In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-30,France,129729.0,0.0,24538.0,0.0,0.0,0.0
0,2020-04-30,China,84037.0,0.0,4668.0,0.0,0.0,0.0
0,2020-04-30,Italy,205632.0,0.0,28051.0,0.0,0.0,0.0
0,2020-04-30,Spain,213901.0,0.0,24895.0,0.0,0.0,0.0
0,2020-04-30,United States,1063552.0,0.0,63397.0,0.0,0.0,0.0
0,2020-04-30,World,3207159.0,0.0,235249.0,0.0,0.0,0.0
0,2020-04-30,United Kingdom,168985.0,0.0,28968.0,0.0,0.0,0.0
0,2020-04-30,Germany,160188.0,0.0,6481.0,0.0,0.0,0.0
0,2020-04-30,Iran,94745.0,0.0,6027.0,0.0,0.0,0.0
0,2020-04-30,Turkey,119677.0,0.0,3881.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-01,France,129729.0,129581.0,24538.0,24376.0,148.0,162.0
0,2020-05-01,China,84037.0,83956.0,4668.0,4637.0,81.0,31.0
0,2020-05-01,Italy,205632.0,205463.0,28051.0,27967.0,169.0,84.0
0,2020-05-01,Spain,213901.0,213435.0,24895.0,24543.0,466.0,352.0
0,2020-05-01,United States,1063552.0,1069826.0,63397.0,63006.0,-6274.0,391.0
0,2020-05-01,World,3207159.0,3214256.0,235249.0,232570.0,-7097.0,2679.0
0,2020-05-01,United Kingdom,168985.0,171253.0,28968.0,26771.0,-2268.0,2197.0
0,2020-05-01,Germany,160188.0,159119.0,6481.0,6288.0,1069.0,193.0
0,2020-05-01,Iran,94745.0,94640.0,6027.0,6028.0,105.0,-1.0
0,2020-05-01,Turkey,119677.0,120204.0,3881.0,3174.0,-527.0,707.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")